download de pdf usando requests lendo pdf direto do buffer da memoria

In [41]:
# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')

# with open('novo_test.pdf','wb') as arqv:
#     arqv.write(file.content)

# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')
# pdf = PyPDF2.PdfFileReader(io.BytesIO(file.content))
# pdf.numPages

In [181]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import re
from multiprocessing.pool import ThreadPool
import pandas as pd

In [2]:
base_url = "https://www.dodf.df.gov.br/listar"

In [3]:
import requests

cookies = {
    '_ga': 'GA1.4.1676642850.1569274904',
    '_gid': 'GA1.4.30087383.1569274904',
    'PHPSESSID': 'c8gp0cncq0bshqvnkq5ng3d0g7',
}

headers = {
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Origin': 'https://www.dodf.df.gov.br',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'Accept': '*/*',
    'Referer': 'https://www.dodf.df.gov.br/listar',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
    'Content-Length': '0',
}

In [5]:
def crawler_params(directory):
    params = (('dir', directory),)
    return params

In [19]:
params = crawler_params('2019/09_Setembro/')
response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
response.text

'{"type":"success","result":true,"dir":"2019\\/09_Setembro\\/","data":{"20190927107":"DODF 185 27-09-2019","20190927100":"DODF 070 27-09-2019 EDICAO EXTRA","20190926100":"DODF 184 26-09-2019","20190925106":"DODF 069 25-09-2019 EDICAO EXTRA","20190925100":"DODF 183 25-09-2019","20190924100":"DODF 182 24-09-2019","20190923100":"DODF 181 23-09-2019","20190920105":"DODF 180 20-09-2019","20190920100":"DODF 068 20-09-2019 EDICAO EXTRA","20190919103":"DODF 179 19-09-2019","20190919100":"DODF 067 19-09-2019 EDICAO EXTRA","20190918100":"DODF 178 18-09-2019","20190917100":"DODF 177 17-09-2019","20190916109":"DODF 176 16-09-2019","20190916100":"DODF 066 16-09-2019 EDICAO EXTRA","20190913101":"DODF 065 13-09-2019 EDICAO EXTRA","20190913100":"DODF 175 13-09-2019","20190912100":"DODF 174 12-09-2019","20190911104":"DODF 064 11-09-2019 EDICAO EXTRA","20190911100":"DODF 173 11-09-2019","20190910100":"DODF 172 10-09-2019","20190909110":"DODF 063 09-09-2019 EDICAO EXTRA","20190909100":"DODF 171 09-09-201

In [131]:
params = crawler_params('2019/09_Setembro/DODF 070 27-09-2019 EDICAO EXTRA')
response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
response.text

'{"type":"success","result":true,"dir":"2019\\/09_Setembro\\/DODF 070 27-09-2019 EDICAO EXTRA","data":["2019\\/09_Setembro\\/DODF 070 27-09-2019 EDICAO EXTRA\\/DODF 070 27-09-2019 EDICAO EXTRA.pdf"]}'

In [132]:
response.json()

{'type': 'success',
 'result': True,
 'dir': '2019/09_Setembro/DODF 070 27-09-2019 EDICAO EXTRA',
 'data': ['2019/09_Setembro/DODF 070 27-09-2019 EDICAO EXTRA/DODF 070 27-09-2019 EDICAO EXTRA.pdf']}

In [133]:
string = response.json()['data'][0]

In [134]:
string

'2019/09_Setembro/DODF 070 27-09-2019 EDICAO EXTRA/DODF 070 27-09-2019 EDICAO EXTRA.pdf'

In [156]:
first_param, second_param, third_param, fourth_param = re.sub(pattern=r"\s", repl="%20",string=string).split('/')

In [153]:
requests.get("https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta="+first_param+"|"+second_param+"|"+third_param+"|&arquivo="+fourth_param)

In [200]:
def get_url_to_request(string):
    st_param, nd_param, rd_param, th_param = \
    re.sub(pattern=r"\s", repl="%20",string=string).split('/')
    return th_param, "https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta="+st_p+"|"+nd_p+"|"+rd_p+"|&arquivo="+th_p

In [201]:
def get_file(th_param, url):
    file = requests.get(url)
    with open(th_param,'wb') as arqv:
        arqv.write(file.content)

In [157]:
params = (('dir', '/'),)
response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
years = json.loads(response.text)['data'][:4]
years = list(map(lambda x:str(x),years))

all_docs = []
for year in years:
    params = (('dir', year),)
    response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
    months = json.loads(response.text)['data']
    for month in months:
        params = (('dir', year+'/'+month),)
        response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
        docs = json.loads(response.text)['data']
        docs = list(docs.values())
        for doc in docs:
            params = (('dir', year+'/'+month+'/'+doc),)
            response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
            all_docs.append(response.json()['data'][0])

In [ ]:
all_docs = [re.sub(pattern=r"(MarÃ§o)", repl="Março", string=doc) for doc in all_docs]

In [178]:
all_docs = pd.DataFrame(all_docs)

In [179]:
all_docs.to_csv('all_documents.csv',index=False)

In [ ]:
ThreadPool(8).map(requests.get(get_url_to_request), all_docs)